In [1]:
import datetime
from IPython.display import display
import itertools
import concurrent.futures
from matplotlib import pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import random
from scipy.optimize import least_squares, minimize
from scipy import linalg
import statistics as stats
import time
import warnings
warnings.filterwarnings('ignore')


# matrix helpers
def _yi(price_series):
    return [np.log(p) for p in price_series]

def _fi(tc, m, time_series):
    return [np.power((tc - t), m) if str(np.power((tc - t), m)) != 'nan' else 0 for t in time_series]
     
def _gi(tc, m, w, time_series):
    return [np.power((tc - t), m) * np.cos(w * np.log(tc - t)) if str(np.power((tc - t), m) * np.cos(w * np.log(tc - t))) != 'nan' else 0 for t in time_series]

def _hi(tc, m, w, time_series):
    return [np.power((tc - t), m) * np.sin(w * np.log(tc - t)) if str(np.power((tc - t), m) * np.sin(w * np.log(tc - t))) != 'nan' else 0 for t in time_series]

def _fi_pow_2(tc, m, time_series):
    return np.power(_fi(tc, m, time_series), 2)

def _gi_pow_2(tc, m, w, time_series):
    return np.power(_gi(tc, m, w, time_series), 2)

def _hi_pow_2(tc, m, w, time_series):
    return np.power(_hi(tc, m, w, time_series), 2)

def _figi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _gi(tc, m, w, time_series))

def _fihi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _hi(tc, m, w, time_series))

def _gihi(tc, m, w, time_series):
    return np.multiply(_gi(tc, m, w, time_series), _hi(tc, m, w, time_series))

def _yifi(tc, m, time_series, price_series):
    return np.multiply(_yi(price_series), _fi(tc, m, time_series))

def _yigi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _gi(tc, m, w, time_series))

def _yihi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _hi(tc, m, w, time_series))

In [2]:
# revised version of the LPPL without φ
# found on page 11 as equation (13)
def lppl(t, tc, m, w, a, b, c1, c2):
    return a + np.power(tc - t, m) * (b + ((c1 * np.cos(w * np.log(tc - t))) + (c2 * np.sin(w * np.log(tc - t)))))


# finds the least square difference
def func_restricted(x, *args):
    
    tc = x[0]
    m  = x[1]
    w  = x[2]
    
    data_series = args[0]
       
    lin_vals = matrix_equation(tc, m, w, data_series)

    a  = lin_vals[0] 
    b  = lin_vals[1]
    c1 = lin_vals[2] 
    c2 = lin_vals[3]

    delta = [lppl(t, tc, m, w, a, b, c1, c2) for t in data_series[0]]
    delta = np.subtract(delta, data_series[1])
    delta = np.power(delta, 2)
    return np.sum(delta)



# solve the matrix equation
def matrix_equation(tc, m, w, data_series):
    time_series = data_series[0]
    price_series = data_series[1]
    N  = len(price_series)
    
    #--------------------------------
    fi = sum(_fi(tc, m, time_series))
    gi = sum(_gi(tc, m, w, time_series))
    hi = sum(_hi(tc, m, w, time_series))

    #--------------------------------
    fi_pow_2 = sum(_fi_pow_2(tc, m, time_series))
    gi_pow_2 = sum(_gi_pow_2(tc, m, w, time_series))
    hi_pow_2= sum(_hi_pow_2(tc, m, w, time_series))

    #--------------------------------
    figi = sum(_figi(tc, m, w, time_series))
    fihi = sum(_fihi(tc, m, w, time_series))
    gihi = sum(_gihi(tc, m, w, time_series))

    #--------------------------------
    yi = sum(_yi(price_series))
    yifi = sum(_yifi(tc, m, time_series, price_series))
    yigi = sum(_yigi(tc, m, w, time_series, price_series))
    yihi = sum(_yihi(tc, m, w, time_series, price_series))
    
    #--------------------------------
    matrix_1 = np.matrix([
        [N,  fi,       gi,       hi      ],
        [fi, fi_pow_2, figi,     fihi    ],
        [gi, figi,     gi_pow_2, gihi    ],
        [hi, fihi,     gihi,     hi_pow_2]
    ])
    
    matrix_2 = np.matrix([
        [yi],
        [yifi],
        [yigi],
        [yihi]
    ])

    try:
        product = linalg.solve(matrix_1, matrix_2)
        return [i[0] for i in product]

    except Exception as e:
        print(e)

        
def calc_lppls_confidence(SYMBOL, ds_lppls_confidence_fits):
    true_count = 0
    total_count = len(ds_lppls_confidence_fits)
    for i in ds_lppls_confidence_fits:
        if i[SYMBOL]['ds_lppls_confidence'] == True:
            true_count = true_count + 1
            
    print('{}/{}={}'.format(true_count, total_count, true_count/total_count))
    return true_count/total_count
        
        
"""
upperbound = 126 # ~6 months (in trading days)
lowerbound = 21 # ~1 month (in trading days)
interval = 5
"""       
def compute_ds_lppls_confidence(df, SYMBOL, upperbound=126, lowerbound=21, interval=5):
    
    ds_lppls = []
    number_of_fitting_windows = int((upperbound - lowerbound)/interval)
    
    for i in range(number_of_fitting_windows):

        tLen = upperbound-(i*interval)

        print('length of data: {}'.format(tLen))
        tradings_days_data = df.tail(tLen)
        time = np.linspace(0, tLen-1, tLen)
        price = np.array([tradings_days_data[SYMBOL][i] for i in range(len(tradings_days_data[SYMBOL]))])
        data_series = np.array([time, price])

        MAX_SEARCHES = 5
        SEARCH_COUNT = 0

        while SEARCH_COUNT < MAX_SEARCHES:

            # set limits for non-linear params
            bounds = [
                (tLen-(tLen*0.2), tLen+(tLen*0.2)),    # Critical Time + or - .2
                (0.1, 0.9),                            # m : 0.1 ≤ m ≤ 0.9
                (6, 13),                               # ω : 6 ≤ ω ≤ 13     
            ]

            # randomly choose vals for non-linear params 
            non_lin_vals = [random.uniform(a[0], a[1]) for a in bounds]

            tc = non_lin_vals[0]
            m  = non_lin_vals[1] 
            w  = non_lin_vals[2]

            # params to pass to scipy.optimize
            seed = np.array([tc, m, w])

            # scipy optimize minimize
            try:

                # Nelder-Mead
                cofs = minimize(
                    args=(data_series, bounds), 
                    fun=func_restricted, 
                    method='Nelder-Mead', 
#                     options={
#                         'adaptive': True
#                     },
                    x0=seed
                )

    #             # Levenberg-Marquardt
    #             cofs = least_squares(
    #                 args=(data_series),
    #                 bounds=bounds,
    #                 fun=func_restricted,
    #                 method='lm',
    #                 x0=seed
    #             )

                if cofs.success:
                    print('minimize ran succsessfully in {} search(es)'.format(SEARCH_COUNT+1))
                    # determine if it falls in range:

                    tc = cofs.x[0]
                    m =  cofs.x[1]
                    w =  cofs.x[2]

                    tc_in_range = tLen-(tLen*0.05) < tc < tLen+(tLen*0.1)
                    m_in_range  = 0.01 < m < 1.2
                    w_in_range  = 2 < w < 25
#                     n_oscillation = ((w/2)*np.log(abs((tc - (i*5))/(tLen)))) > 2.5

#                     if (tc_in_range and m_in_range and w_in_range and n_oscillation):
                    if (tc_in_range and m_in_range and w_in_range):
                        ds_lppls_confidence = True
                    else: 
                        ds_lppls_confidence = False

                    ds_lppls.append({SYMBOL: {
                        'ds_lppls_confidence': ds_lppls_confidence,
                        'cof': cofs.x,
                    }})
                    break

                else:
                    SEARCH_COUNT = SEARCH_COUNT + 1
                    print('minimize failed to find a solution, trying again')
                    
            except Exception as e:
                print('minimize failed: {}'.format(e))
                SEARCH_COUNT = SEARCH_COUNT + 1
                
        if SEARCH_COUNT >= MAX_SEARCHES:
            # no solution found in 5 tries, so just add this and move one
            print('minimize failed in allotted attempts (5)')
            ds_lppls.append({SYMBOL: {
                'ds_lppls_confidence': False,
                'cof': None,
                'max_searches_exceeded': True
            }})
    
    return calc_lppls_confidence(SYMBOL, ds_lppls)


def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)


def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

In [3]:
data = pd.read_csv("../data/backtest-2.csv", index_col="date")
data.fillna(method='ffill')

# # print(len(data))
# # display(data.head())
# # display(data.tail())

SYMBOL = 'WMT'
# start = time.time()
# ds_lppls_confidence = compute_ds_lppls_confidence(data, SYMBOL, upperbound=126, lowerbound=21, interval=5)
# end = time.time()
# duration_of_fit = end - start

# print(duration_of_fit)
# print(ds_lppls_confidence)


master_data_len = len(data)
window_len = 126
ground_to_cover = master_data_len-window_len

rolling_ds_lppls_confidence = []

for i in range(ground_to_cover):
    window = data[SYMBOL].iloc[i:window_len+i]
#     print(i)
#     print(len(window))
#     print(window)
    
    start = time.time()
    ds_lppls_confidence = compute_ds_lppls_confidence(data, SYMBOL, upperbound=126, lowerbound=21, interval=5)
    end = time.time()
    
    duration_of_fit = end - start
    rolling_ds_lppls_confidence.append({
        'val': ds_lppls_confidence,
        'duration_of_fit': duration_of_fit,
        'date': window.tail(1).index.tolist()[0],
        'price': window.tail(1).tolist()[0]
    })
    print(rolling_ds_lppls_confidence)

# workers = 12
# pool = multiprocessing.Pool(processes=workers)
# result = pool.map(compute_ds_lppls_confidence, [(d, func, kwargs) for d in np.array_split(df, workers)])
# pool.close()

length of data: 126
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize ran succsessfully in 3 search(es)
length of data: 121
minimize failed to find a solution, trying again
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed to find a solution, trying again
array must not contain infs or NaNs
minimize failed: 'NoneType' object is not subscriptable
minimize failed to find a solution, trying again
minimize failed in allotted attempts (5)
length of data: 116
minimize ran succsessfully in 1 search(es)
length of data: 111
minimize ran succsessfully in 1 search(es)
length of data: 106
minimize ran succsessfully in 1 search(es)
length of data: 101
minimize ran succsessfully in 1 search(es)
length of data: 96
minimize ran succsessfully in 1 search(es)
length of data: 91
minimize failed to find a solution, trying again
minimize ran succsessfully 

minimize ran succsessfully in 1 search(es)
length of data: 51
minimize failed to find a solution, trying again
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed in allotted attempts (5)
length of data: 46
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed to find a solution, trying again
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed to find a solution, trying again
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed in allotted attempts (5)
length of data: 41
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not s

minimize failed to find a solution, trying again
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed in allotted attempts (5)
length of data: 36
minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
length of data: 31
minimize ran succsessfully in 1 search(es)
length of data: 26
minimize ran succsessfully in 1 search(es)
0/21=0.0
[{'val': 0.0, 'duration_of_fit': 162.76369190216064, 'date': '2018-09-07', 'price': 94.7998}, {'val': 0.0, 'duration_of_fit': 169.05714583396912, 'date': '2018-09-10', 'price': 95.8583}, {'val': 0.0, 'duration_of_fit': 168.2596893310547, 'date': '2018-09-11', 'price': 95.6011}, {'val': 0.0, 'duration_of_fit': 184.42379808425903, 'date': '2018-09-12', 'price': 94.9383}, {'val': 0.0, 'duration_of_fit': 214.95388317108154, 'date': '2018-09-13', 'price': 94.0975}]
length of data: 126
minimize failed to find a sol

minimize ran succsessfully in 1 search(es)
length of data: 101
minimize ran succsessfully in 1 search(es)
length of data: 96
minimize ran succsessfully in 1 search(es)
length of data: 91
minimize failed to find a solution, trying again
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed to find a solution, trying again
minimize ran succsessfully in 4 search(es)
length of data: 86
minimize ran succsessfully in 1 search(es)
length of data: 81
minimize ran succsessfully in 1 search(es)
length of data: 76
minimize ran succsessfully in 1 search(es)
length of data: 71
minimize ran succsessfully in 1 search(es)
length of data: 66
minimize ran succsessfully in 1 search(es)
length of data: 61
minimize ran succsessfully in 1 search(es)
length of data: 56
minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
length of data: 51
minimize failed to find a solution, trying again
minimize failed to find a solution, trying ag

minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
length of data: 61
minimize ran succsessfully in 1 search(es)
length of data: 56
minimize ran succsessfully in 1 search(es)
length of data: 51
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed to find a solution, trying again
minimize failed to find a solution, trying again
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed in allotted attempts (5)
length of data: 46
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed to find a solution, trying again
array must not contain infs or NaNs
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize f

...

minimize failed to find a solution, trying again
minimize ran succsessfully in 3 search(es)
length of data: 26
minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
0/21=0.0
[{'val': 0.0, 'duration_of_fit': 162.76369190216064, 'date': '2018-09-07', 'price': 94.7998}, {'val': 0.0, 'duration_of_fit': 169.05714583396912, 'date': '2018-09-10', 'price': 95.8583}, {'val': 0.0, 'duration_of_fit': 168.2596893310547, 'date': '2018-09-11', 'price': 95.6011}, {'val': 0.0, 'duration_of_fit': 184.42379808425903, 'date': '2018-09-12', 'price': 94.9383}, {'val': 0.0, 'duration_of_fit': 214.95388317108154, 'date': '2018-09-13', 'price': 94.0975}, {'val': 0.0, 'duration_of_fit': 183.38385272026062, 'date': '2018-09-14', 'price': 93.5732}, {'val': 0.0, 'duration_of_fit': 122.41044807434082, 'date': '2018-09-17', 'price': 93.8007}, {'val': 0.0, 'duration_of_fit': 139.55217385292053, 'date': '2018-09-18', 'price': 94.4041}, {'val': 0.0, 'duration_of_fit': 219.58280396

minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
length of data: 111
minimize ran succsessfully in 1 search(es)
length of data: 106
minimize ran succsessfully in 1 search(es)
length of data: 101
minimize failed to find a solution, trying again
minimize failed to find a solution, trying again
minimize ran succsessfully in 3 search(es)
length of data: 96
minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
length of data: 91
minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
length of data: 86
minimize ran succsessfully in 1 search(es)
length of data: 81
minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
length of data: 76
minimize ran succsessfully in 1 search(es)
length of data: 71
minimize failed to find a solution, trying again
minimize ran succsessfully in 2 search(es)
length of data: 66
minimize ran succsessfully in 1 se

Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
Matrix is singular.
minimize failed: 'NoneType' object is not subscriptable
minimize failed to find a solution, trying again
minimize ran succsessfully in 4 search(es)
length of data: 26
minimize ran succsessfully in 1 search(es)
0/21=0.0
[{'val': 0.0, 'duration_of_fit': 162.76369190216064, 'date': '2018-09-07', 'price': 94.7998}, {'val': 0.0, 'duration_of_fit': 169.05714583396912, 'date': '2018-09-10', 'price': 95.8583}, {'val': 0.0, 'duration_of_fit': 168.2596893310547, 'date': '2018-09-11', 'price': 95.6011}, {'val': 0.0, 'duration_of_fit': 184.42379808425903, 'date': '2018-09-12', 'price': 94.9383}, {'val': 0.0, 'duration_of_fit': 214.95388317108154, 'date': '2018-09-13', 'price': 94.0975}, {'val': 0.0, 'duration_of_fit': 183.38385272026062, 'date': '2018-09-14', 'price': 93.5732}, {'val': 0.0, 'duration_of_fit': 122.41044807434082, 'date': '2018-09-17', 'price': 93.8007}, {'val': 0.0, 'duration_of_fit': 13

minimize ran succsessfully in 1 search(es)
length of data: 121
minimize failed to find a solution, trying again
minimize failed to find a solution, trying again
minimize failed to find a solution, trying again
minimize failed to find a solution, trying again
array must not contain infs or NaNs
minimize failed: 'NoneType' object is not subscriptable
minimize failed in allotted attempts (5)
length of data: 116
minimize failed to find a solution, trying again
minimize failed to find a solution, trying again
minimize ran succsessfully in 3 search(es)
length of data: 111
minimize ran succsessfully in 1 search(es)
length of data: 106
minimize ran succsessfully in 1 search(es)
length of data: 101
minimize ran succsessfully in 1 search(es)
length of data: 96
minimize ran succsessfully in 1 search(es)
length of data: 91
minimize ran succsessfully in 1 search(es)
length of data: 86
minimize ran succsessfully in 1 search(es)
length of data: 81
minimize ran succsessfully in 1 search(es)
length of 

KeyboardInterrupt: 

In [ ]:
# print fits from lppls confidence searches
for i in range(number_of_fitting_windows):
    tLen = 126-(i*5)
    DataSeries = data.tail(tLen)
    time2 = DataSeries.index.tolist()
    time2 = np.linspace(0, len(time2)-1, len(time2))
    price = DataSeries[SYMBOL].tolist()
    DataSeries = [time2, price]

    if ds_lppls[i][SYMBOL]['cof'] is not None:
        tc = ds_lppls[i][SYMBOL]['cof'][0]
        m =  ds_lppls[i][SYMBOL]['cof'][1]
        w =  ds_lppls[i][SYMBOL]['cof'][2]

        lin_vals = matrix_equation(tc, m, w, DataSeries)

        a  = lin_vals[0] 
        b  = lin_vals[1]
        c1 = lin_vals[2] 
        c2 = lin_vals[3]
        lppl_fit = [lppl(t, tc, m, w, a, b, c1, c2) for t in DataSeries[0]]
        price_data = DataSeries[1]

        data2 = pd.DataFrame({
            'Date': DataSeries[0],
            'LPPLS Fit': lppl_fit,
            SYMBOL: np.log(price_data),
        })
        data2 = data2.set_index('Date')
        data2.plot(figsize=(14,8))

In [ ]:
def main(time_range, s):
    solutions = []
    for i in range(time_range):
        
        
        #     with concurrent.futures.ProcessPoolExecutor() as executor:
        #         for foo in executor.map(compute_ds_lppls_confidence, time_range, s):
        #             solutions.append(foo)

        foo = compute_ds_lppls_confidence(i, s)
        solutions.append(foo)
        
    true_count = 0
    total_count = len(solutions)
    for i in solutions:
        if i["ds_lppls_confidence"] == True:
            true_count = true_count + 1

#     print("{}/{}={}".format(true_count,total_count,true_count/total_count))
    return (true_count/total_count)